In [1]:
import copy
import random
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import gym

Let's get some information about environment

In [4]:
env = gym.make("MountainCar-v0")

In [5]:
print(env.action_space)
print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)

Discrete(3)
Box(2,)
[0.6  0.07]
[-1.2  -0.07]


In [9]:
state = env.reset()

for time in range(10):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print(f"time: {time}, action: {action}, observation: {observation}, reward: {reward}")

    if done:
        print("Episode finished after {} timesteps".format(time+1))
        break
env.close()

time: 0, action: 0, observation: [-0.40829502 -0.00186075], reward: -1.0
time: 1, action: 0, observation: [-0.41200341 -0.00370839], reward: -1.0
time: 2, action: 2, observation: [-0.41553321 -0.0035298 ], reward: -1.0
time: 3, action: 1, observation: [-0.41985938 -0.00432617], reward: -1.0
time: 4, action: 2, observation: [-0.4239511  -0.00409172], reward: -1.0
time: 5, action: 2, observation: [-0.4277791  -0.00382799], reward: -1.0
time: 6, action: 1, observation: [-0.43231588 -0.00453679], reward: -1.0
time: 7, action: 0, observation: [-0.43852877 -0.00621288], reward: -1.0
time: 8, action: 0, observation: [-0.44637277 -0.007844  ], reward: -1.0
time: 9, action: 0, observation: [-0.4557908  -0.00941803], reward: -1.0


It's clear that provided task can't be solved with simple policy of taking random actions. Let's try to learn optimal policy with deep Q learning algorithm. More precisely, we will learn Q values and than take based on that take 